## Load library nltk, numpy, pandas, json

In [1]:
import nltk
nltk.download('punkt')
nltk.download('wordnet')
from nltk.stem import WordNetLemmatizer
lemmatizer = WordNetLemmatizer()
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
import string
import random
import pandas as pd
import numpy as np
import warnings
import requests
import re
from string import punctuation
warnings.filterwarnings('ignore')

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Hp\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [11]:
pip install --upgrade gensim

Requirement already up-to-date: gensim in c:\users\hp\anaconda3\lib\site-packages (3.8.2)
Note: you may need to restart the kernel to use updated packages.


## Load doc2vec from gensim

In [12]:
import gensim
from gensim import corpora, models, similarities
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
from gensim.utils import simple_preprocess
import multiprocessing
import os

## Import the Text file on Coronavirus

In [62]:
f = open("E:\Blog\Chatbot\Covid_new_text.txt", 'r', errors = "ignore")

## Treat the Raw text file

In [63]:
raw = f.read()
raw = raw.lower()

## Doing cleaning of the document
### Remove special characters

In [64]:
review_text = re.sub(r"[^A-Za-z0-9(),!.?\'`]", " ", raw )
review_text = re.sub(r"\'s", " 's ", review_text )
review_text = re.sub(r"\'ve", " 've ", review_text )
review_text = re.sub(r"n\'t", " 't ", review_text )
review_text = re.sub(r"\'re", " 're ", review_text )
review_text = re.sub(r"\'d", " 'd ", review_text )
review_text = re.sub(r"\'ll", " 'll ", review_text )
#review_text = re.sub(r"\(", " ", review_text )
#review_text = re.sub(r"\)", " ", review_text )
review_text = re.sub(r"\?", " ", review_text )
review_text = re.sub(r"\s{2,}", " ", review_text )

## Sentence Tokenizing

In [65]:
sent_tokens = nltk.sent_tokenize(review_text)

## Remove STOP WORDS

In [6]:
stop_words = set(stopwords.words('english'))
all_sentence =[]
new_sentence = []

for s in range(0,len(sent_tokens)-1):
    word_tokens = word_tokenize(sent_tokens[s])
    for w in word_tokens:
            if w not in stop_words:
                new_sentence.append(w)

    all_sentence.append(new_sentence)
    new_sentence = []

In [7]:
model_sentence = [[' '.join(i)] for i in all_sentence]

In [8]:
model_sentence

[['coronaviruses large family viruses may cause illness animals humans .'],
 ['humans , several coronaviruses known cause respiratory infections ranging common cold severe diseases middle east respiratory syndrome ( mers ) severe acute respiratory syndrome ( sars ) .covid 19 infectious disease caused recently discovered corona virus .'],
 ['new virus disease unknown outbreak began wuhan , china , december 2019. common symptoms covid 19 fever , tiredness , dry cough .'],
 ['patients may aches pains , nasal congestion , runny nose , sore throat diarrhea .'],
 ['symptoms usually mild begin gradually .'],
 ["people become infected develop symptoms 't feel unwell ."],
 ['people ( 80 ) recover disease without needing special treatment .'],
 ['around 1 every 6 people gets covid 19 becomes seriously ill develops difficulty breathing .'],
 ['older people , underlying medical problems like high blood pressure , heart problems diabetes , likely develop serious illness .'],
 ['people fever , cough

## Tagging the document for each of the sentence in the doc

In [66]:
documents = [TaggedDocument(doc, [i]) for i, doc in enumerate(sent_tokens)]

In [67]:
documents

[TaggedDocument(words='coronaviruses are a large family of viruses which may cause illness in animals or humans.', tags=[0]),
 TaggedDocument(words='in humans, several coronaviruses are known to cause respiratory infections ranging from the common cold to more severe diseases such as middle east respiratory syndrome (mers) and severe acute respiratory syndrome (sars).', tags=[1]),
 TaggedDocument(words='the most recently discovered coronavirus causes coronavirus disease covid 19. what is covid 19 covid 19 is the infectious disease caused by the most recently discovered corona virus.', tags=[2]),
 TaggedDocument(words='this new virus and disease were unknown before the outbreak began in wuhan, china, in december 2019. what are the symptoms of covid 19 the most common symptoms of covid 19 are fever, tiredness, and dry cough.', tags=[3]),
 TaggedDocument(words='some patients may have aches and pains, nasal congestion, runny nose, sore throat or diarrhea.', tags=[4]),
 TaggedDocument(words

## Run the Doc2Vec model

In [68]:
max_epochs = 1000
vec_size = 80
alpha = 0.025

model = Doc2Vec(size=vec_size,
                alpha=alpha, 
                min_alpha=0.00025,
                min_count=1,
                dm =1,
                dbow_words=1)



In [69]:
model.build_vocab(documents)

In [70]:
for epoch in range(max_epochs):
    #print('iteration {0}'.format(epoch))
    model.train(documents,
                total_examples=model.corpus_count,
                epochs=model.iter)
    # decrease the learning rate
    model.alpha -= 0.0002
    # fix the learning rate, no decay
    model.min_alpha = model.alpha

model.save("d2v.model")
print("Model Saved")

Model Saved


## Greetings for the BOT

In [71]:
greeting_inputs = ("hey", "good morning", "good evening", "morning", "evening", "hi", "whatsup")
greeting_responses = ["hey", "hey hows you?", "hello, how you doing", "hello", "Welcome, I am good and you"]

def greeting(sentence):
    for word in sentence.split():
        if word.lower() in greeting_inputs:
            return random.choice(greeting_responses)

## Defining a function to get response of user

In [72]:
def response_(user_response):
    chatbot_return=''
    test_data = word_tokenize(user_response.lower())
    v1 = model.infer_vector(test_data)
    
    ## Find similar sentence to the user response
    similar_doc = model.docvecs.most_similar([v1])
    #print(similar_doc)
    if(similar_doc==0):
        chatbot_return = chatbot_return + "Sorry! I don't understand you"
        return chatbot_return
    else:
        #print(int(similar_doc[0][0]))
        retunr_string = documents[int(similar_doc[0][0])][0]
        #print(retunr_string)
        listToStr = ' '.join([str(elem) for elem in retunr_string])
        chatbot_return = chatbot_return + retunr_string
        return chatbot_return

In [73]:
flag = True
print("ChatBot: I can share some knowlegde about Coronavirus")
while(flag==True):
    user_response = input()
    user_response = user_response.lower()
    if(user_response=='thanks' or user_response =='thank you'):
            flag = False
            print("Chatbot: You are welcome")
    else:
        if(greeting(user_response)!=None):
                print("ChatBot:" + greeting(user_response))
        else:
                print("Chatbot :", end = "")
                print(response_(user_response))

ChatBot: I can share some knowlegde about Coronavirus
Hi
ChatBot:hey
Hey
ChatBot:hey
What are coronaviruses?
Chatbot :refrain from smoking and other activities that weaken the lungs.
COVID-19?
Chatbot :is there a vaccine drug or treatment for covid 19 not yet.
Thanks
Chatbot: You are welcome
